In [2]:
import numpy as np
import sys
sys.path.append("/Users/utilizador/Documents/GitHub/si/src")

from si.neural_networks.layers import Layer

A classe Dropout implementa a técnica de regularização chamada dropout, amplamente utilizada em redes neurais. O dropout é projetado para reduzir o overfitting durante o treinamento, desativando aleatoriamente neurônios (definindo seus valores como 0) em cada iteração de treinamento. Durante a inferência (avaliação ou predição), o dropout é desativado e todos os neurônios participam da predição.

__Dropout:__
Durante o treinamento, cada neurônio tem uma probabilidade definida de ser "desativado" (ou seja, definido como 0). Isso força a rede a aprender representações redundantes e mais robustas.

__Vantagens:__
Reduz o overfitting.
Aumenta a generalização ao impedir que a rede dependa excessivamente de neurônios individuais.

__Desvantagens:__
Introduz aleatoriedade durante o treinamento.
Não garante melhoria de desempenho em todos os cenários.

__init__:
Inicializa a camada com uma taxa de dropout definida. Valida se o valor está no intervalo válido.

__forward_propagation:__

Durante o treinamento:
Gera uma máscara binária para desativar neurônios.
Aplica o escalonamento aos neurônios restantes.
Durante a inferência:
Nenhuma alteração é feita no input.

__backward_propagation:__
Propaga o erro para trás apenas pelos neurônios que estavam ativos durante o forward propagation.

__output_shape:__
Retorna o formato do input, já que o dropout não altera a forma dos dados.

__parameters:__
Retorna 0, indicando que a camada de dropout não possui parâmetros treináveis.

In [3]:
class Dropout(Layer):
    def __init__(self, probability: float):
        """
        Initializes the Dropout layer.
        
        Parameters:
        - probability (float): Dropout rate, between 0 and 1.
        """
        if not (0 <= probability <= 1):
            raise ValueError("Dropout probability must be between 0 and 1.")
        self.probability = probability
        self.mask = None
        self.input = None
        self.output = None

    def forward_propagation(self, input: np.ndarray, training: bool = True) -> np.ndarray:
        """
        Perform forward propagation through the dropout layer.
        
        Parameters:
        - input (np.ndarray): The input array.
        - training (bool): Whether we are in training mode or not.
        
        Returns:
        - np.ndarray: The output array after applying dropout (or unchanged input during inference).
        """
        self.input = input
        if training:
            scaling_factor = 1 / (1 - self.probability)
            # Gera a máscara binária, onde cada neurônio tem probabilidade (1 - probability) de ser mantido
            self.mask = np.random.binomial(1, 1 - self.probability, size=input.shape)
            self.output = input * self.mask * scaling_factor
        else:
            self.output = input
        return self.output

    def backward_propagation(self, output_error: np.ndarray) -> np.ndarray:
        """
        Perform backward propagation through the dropout layer.
        
        Parameters:
        - output_error (np.ndarray): The output error of the layer.
        
        Returns:
        - np.ndarray: The error propagated back through the layer.
        """
        return output_error * self.mask # Propaga o erro pelos neurônios ativos

    def output_shape(self) -> tuple:
        """
        Returns the shape of the input (dropout does not change the shape).
        
        Returns:
        - tuple: The shape of the input.
        """
        return self.input.shape if self.input is not None else None

    @property
    def parameters(self) -> int:
        """
        Returns the number of parameters in the layer.
        
        Returns:
        - int: Always 0 for the dropout layer.
        """
        return 0


In [4]:
# Criar uma instância da camada Dropout
dropout_layer = Dropout(probability=0.5)

# Gerar entrada aleatória
np.random.seed(42)  # Para reprodutibilidade
input_data = np.random.rand(5, 5)  # Matriz 5x5 com valores aleatórios

# Propagação para frente em modo de treinamento
output_train = dropout_layer.forward_propagation(input_data, training=True)

# Propagação para frente em modo de inferência
output_inference = dropout_layer.forward_propagation(input_data, training=False)

# Exibir os resultados
print("Input Data:")
print(input_data)
print("\nOutput in Training Mode (Dropout Applied):")
print(output_train)
print("\nOutput in Inference Mode (No Dropout Applied):")
print(output_inference)


Input Data:
[[0.37454012 0.95071431 0.73199394 0.59865848 0.15601864]
 [0.15599452 0.05808361 0.86617615 0.60111501 0.70807258]
 [0.02058449 0.96990985 0.83244264 0.21233911 0.18182497]
 [0.18340451 0.30424224 0.52475643 0.43194502 0.29122914]
 [0.61185289 0.13949386 0.29214465 0.36636184 0.45606998]]

Output in Training Mode (Dropout Applied):
[[0.74908024 0.         1.46398788 1.19731697 0.        ]
 [0.31198904 0.         0.         1.20223002 1.41614516]
 [0.04116899 0.         0.         0.42467822 0.        ]
 [0.         0.         0.         0.86389004 0.        ]
 [1.22370579 0.         0.5842893  0.73272369 0.        ]]

Output in Inference Mode (No Dropout Applied):
[[0.37454012 0.95071431 0.73199394 0.59865848 0.15601864]
 [0.15599452 0.05808361 0.86617615 0.60111501 0.70807258]
 [0.02058449 0.96990985 0.83244264 0.21233911 0.18182497]
 [0.18340451 0.30424224 0.52475643 0.43194502 0.29122914]
 [0.61185289 0.13949386 0.29214465 0.36636184 0.45606998]]
